<h3> prøver å gjenskape henning sin 143.</h3>
<p><p>

In [1]:
%load_ext autoreload

In [2]:
from sklearn.model_selection import KFold
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

from sklearn.metrics import mean_absolute_error
from xgboost import XGBRFRegressor
import pandas as pd
import numpy as np

/Users/miksx/.pyenv/versions/3.11.4/envs/machinelearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload
from pipeline import Pipeline
pipin = Pipeline()

Current working directory: /Users/miksx/GitHub/Forest-Gump/mikael/optuna


In [4]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [5]:
# remove_consecutive_measurments
# 6/24
# dette skal prøve å gjenskape henning sin
df1_0 = pipin.remove_consecutive_measurments_new(df1_0, 4, 12)
df2_0 = pipin.remove_consecutive_measurments_new(df2_0, 4, 12)
df3_0 = pipin.remove_consecutive_measurments_new(df3_0, 4, 12)

In [6]:
df1_0 = pipin.drop_columns(df1_0)
df2_0 = pipin.drop_columns(df2_0)
df3_0 = pipin.drop_columns(df3_0)
print(df1_0.shape, df2_0.shape, df3_0.shape)

(23231, 47) (15838, 47) (13238, 47)


In [7]:
df1_0.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,estimated,observed,day_of_year,hour,pv_measurement
0,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,0.000000,...,0.0,286.225006,100.000000,40386.476562,3.600,0.0,1.0,153.0,22.0,0.00
1,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,0.000000,...,0.0,286.899994,100.000000,33770.648438,3.350,0.0,1.0,153.0,23.0,0.00
2,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,0.000000,...,0.0,286.950012,100.000000,13595.500000,3.050,0.0,1.0,154.0,0.0,0.00
3,8.425,1.21800,1283.425049,208.649994,0.75,1283.425049,0.0,281.674988,0.300,526.775024,...,0.0,286.750000,100.000000,2321.850098,2.725,0.0,1.0,154.0,1.0,0.00
4,8.950,1.21800,1003.500000,32468.150391,23.10,1003.500000,0.0,282.500000,11.975,22068.949219,...,0.0,286.450012,99.224998,11634.799805,2.550,0.0,1.0,154.0,2.0,19.36


In [8]:
def get_objective(X=None, y=None):
    def objective(trial):
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        objective_list_reg = ["reg:squarederror", "reg:squaredlogerror"]
        tree_method = ["approx", "hist"]

        params = {
            "eval_metric": "mae",
            "objective": trial.suggest_categorical("objective", objective_list_reg),
            "tree_method": trial.suggest_categorical("tree_method", tree_method),
            "max_depth": trial.suggest_int("max_depth", 3, 10),  # Adjust the range
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
            "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),  # Increase the range
            "gamma": trial.suggest_float("gamma", 0.1, 1.0),  # Increase the lower bound
            "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
            "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range
            "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 1.0),
            "random_state": trial.suggest_int("random_state", 1, 1000),
        }
        # Initialize a list to store the mean absolute errors for each fold
        mae_list = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
            y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

            print(f"Training Fold {fold}...")
            model = XGBRFRegressor(**params)
            model.fit(X_fold_train, y_fold_train)
            predictions = model.predict(X_fold_val)
            mae_fold = mean_absolute_error(y_fold_val, predictions)
            mae_list.append(mae_fold)
            print(f"Fold {fold} MAE: {mae_fold}")

        # Calculate the mean of the mean absolute errors for all folds
        mae = sum(mae_list) / len(mae_list)
        print(f"Mean MAE for all Folds: {mae}")
        return mae
    return objective

In [9]:
X_1, y_1 = df1_0[[c for c in df1_0.columns if "pv_measurement" != c]], df1_0["pv_measurement"]

In [10]:
study1 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study1.optimize(get_objective(X=X_1,y=y_1 ), n_trials=30)
print("Optimization complete!")

predictor1 = XGBRFRegressor(**study1.best_params).fit(X_1, y_1)


[I 2023-11-09 11:39:04,000] A new study created in memory with name: no-name-0bc9a4f6-1d4c-4365-8832-6e8a2adabc69


Optimizing hyperparameters...
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 542.3241210538304
Training Fold 2...
Fold 2 MAE: 527.6004768792628
Training Fold 3...
Fold 3 MAE: 526.5079243049506
Training Fold 4...
Fold 4 MAE: 536.6648573422115
Training Fold 5...
Fold 5 MAE: 544.4209694216757
Training Fold 6...
Fold 6 MAE: 527.5621305137105
Training Fold 7...
Fold 7 MAE: 528.6335555960648
Training Fold 8...
Fold 8 MAE: 521.1113837718265
Training Fold 9...
Fold 9 MAE: 554.9767970405279
Training Fold 10...


[I 2023-11-09 11:40:47,078] Trial 0 finished with value: 535.0172522711507 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'approx', 'max_depth': 7, 'learning_rate': 0.6030342590056562, 'n_estimators': 871, 'min_child_weight': 2, 'gamma': 0.20627886260773487, 'subsample': 0.85, 'colsample_bytree': 0.95, 'reg_alpha': 0.39411905269249875, 'reg_lambda': 0.6577473704463818, 'random_state': 739}. Best is trial 0 with value: 535.0172522711507.


...

Fold 10 MAE: 342.0094996941936
Mean MAE for all Folds: 339.5908767450837
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 588.1662800521982
Training Fold 2...
Fold 2 MAE: 570.9639473156512
Training Fold 3...
Fold 3 MAE: 573.9813994586236
Training Fold 4...
Fold 4 MAE: 584.3724340862351
Training Fold 5...
Fold 5 MAE: 591.1846611059076
Training Fold 6...
Fold 6 MAE: 575.2485479497232
Training Fold 7...
Fold 7 MAE: 575.4310691856487
Training Fold 8...
Fold 8 MAE: 569.5394543052407
Training Fold 9...
Fold 9 MAE: 601.4668142486059
Training Fold 10...


[I 2023-11-09 12:17:01,763] Trial 29 finished with value: 581.8617490862531 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'approx', 'max_depth': 7, 'learning_rate': 0.5380261024162454, 'n_estimators': 443, 'min_child_weight': 3, 'gamma': 0.30504968909009, 'subsample': 0.8, 'colsample_bytree': 0.65, 'reg_alpha': 0.8696002360726376, 'reg_lambda': 0.6271742548767807, 'random_state': 744}. Best is trial 23 with value: 315.67197458967576.


Fold 10 MAE: 588.2628831546962
Mean MAE for all Folds: 581.8617490862531
Optimization complete!


In [11]:
X_2, y_2 = df2_0[[c for c in df2_0.columns if "pv_measurement" != c]], df2_0["pv_measurement"]

In [12]:
study2 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study2.optimize(get_objective(X=X_2,y=y_2 ), n_trials=30)
print("Optimization complete!")
predictor2 = XGBRFRegressor(**study2.best_params).fit(X_2, y_2)


[I 2023-11-09 12:17:03,491] A new study created in memory with name: no-name-49345c3f-239c-41fc-9d05-d5b82f4e451b
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 138.5997129992479
Training Fold 2...
Fold 2 MAE: 136.33886458576683
Training Fold 3...
Fold 3 MAE: 137.92164584027316
Training Fold 4...
Fold 4 MAE: 140.68485469559945
Training Fold 5...
Fold 5 MAE: 137.44298647432404
Training Fold 6...
Fold 6 MAE: 137.31709833417722
Training Fold 7...
Fold 7 MAE: 141.5450231965709
Training Fold 8...
Fold 8 MAE: 138.56134614016761
Training Fold 9...
Fold 9 MAE: 139.28085297127257
Training Fold 10...


[I 2023-11-09 12:17:41,689] Trial 0 finished with value: 138.73666762287942 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'approx', 'max_depth': 5, 'learning_rate': 0.32297627389351774, 'n_estimators': 532, 'min_child_weight': 8, 'gamma': 0.16887848528002686, 'subsample': 0.85, 'colsample_bytree': 0.85, 'reg_alpha': 0.6082274993749409, 'reg_lambda': 0.10923136237940471, 'random_state': 855}. Best is trial 0 with value: 138.73666762287942.


Fold 10 MAE: 139.67429099139443
Mean MAE for all Folds: 138.73666762287942
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


...

Fold 10 MAE: 76.90482777643491
Mean MAE for all Folds: 75.85967210999846
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 55.9430683715824
Training Fold 2...
Fold 2 MAE: 47.70321686416778
Training Fold 3...
Fold 3 MAE: 51.665555394585716
Training Fold 4...
Fold 4 MAE: 51.681214192953306
Training Fold 5...
Fold 5 MAE: 52.69442383645447
Training Fold 6...
Fold 6 MAE: 53.42255923359569
Training Fold 7...
Fold 7 MAE: 52.39341993834069
Training Fold 8...
Fold 8 MAE: 51.4395870201676
Training Fold 9...
Fold 9 MAE: 53.691445863659794
Training Fold 10...


[I 2023-11-09 13:08:36,755] Trial 29 finished with value: 52.45033959885611 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'hist', 'max_depth': 5, 'learning_rate': 0.9870719964196271, 'n_estimators': 502, 'min_child_weight': 4, 'gamma': 0.16605217583990312, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_alpha': 0.6025622754276665, 'reg_lambda': 0.09878146457174712, 'random_state': 904}. Best is trial 23 with value: 44.97897632718659.


Fold 10 MAE: 53.86890527305375
Mean MAE for all Folds: 52.45033959885611
Optimization complete!


In [13]:
X_3, y_3 = df3_0[[c for c in df3_0.columns if "pv_measurement" != c]], df3_0["pv_measurement"]

In [14]:
study3 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study3.optimize(get_objective(X=X_3,y=y_3 ), n_trials=30)
print("Optimization complete!")
predictor3 = XGBRFRegressor(**study3.best_params).fit(X_3, y_3)


[I 2023-11-09 13:08:53,978] A new study created in memory with name: no-name-c424a940-72c9-42db-af06-e718342f7d0c


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 150.17439839263966
Training Fold 2...
Fold 2 MAE: 142.93217396838247
Training Fold 3...
Fold 3 MAE: 149.12220407417234
Training Fold 4...
Fold 4 MAE: 144.94794801845472
Training Fold 5...
Fold 5 MAE: 154.21661750231175
Training Fold 6...
Fold 6 MAE: 157.15710067168465
Training Fold 7...
Fold 7 MAE: 153.83941892053065
Training Fold 8...
Fold 8 MAE: 155.44798499410516
Training Fold 9...
Fold 9 MAE: 156.62194892663717
Training Fold 10...


[I 2023-11-09 13:11:04,604] Trial 0 finished with value: 152.3466470602913 and parameters: {'objective': 'reg:squaredlogerror', 'tree_method': 'hist', 'max_depth': 10, 'learning_rate': 0.1552576111713436, 'n_estimators': 746, 'min_child_weight': 8, 'gamma': 0.19868933664994717, 'subsample': 0.75, 'colsample_bytree': 0.65, 'reg_alpha': 0.4407663019894997, 'reg_lambda': 0.40621188881437087, 'random_state': 705}. Best is trial 0 with value: 152.3466470602913.


Fold 10 MAE: 159.00667513399466
Mean MAE for all Folds: 152.3466470602913
Training Fold 1...


...

Fold 10 MAE: 48.80868964205103
Mean MAE for all Folds: 49.589294284248375
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3241/2078315071.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 62.45961556691058
Training Fold 2...
Fold 2 MAE: 60.155564347581205
Training Fold 3...
Fold 3 MAE: 63.17095594542506
Training Fold 4...
Fold 4 MAE: 58.155478754763884
Training Fold 5...
Fold 5 MAE: 64.1348829558681
Training Fold 6...
Fold 6 MAE: 63.90207006229736
Training Fold 7...
Fold 7 MAE: 64.10826362344794
Training Fold 8...
Fold 8 MAE: 62.31728073177741
Training Fold 9...
Fold 9 MAE: 65.20710467556978
Training Fold 10...


[I 2023-11-09 13:44:19,106] Trial 29 finished with value: 62.639220692164464 and parameters: {'objective': 'reg:squarederror', 'tree_method': 'approx', 'max_depth': 10, 'learning_rate': 0.7424605372821743, 'n_estimators': 404, 'min_child_weight': 6, 'gamma': 0.4304392016406004, 'subsample': 0.95, 'colsample_bytree': 0.65, 'reg_alpha': 0.4273170386378053, 'reg_lambda': 0.5021074066585709, 'random_state': 4}. Best is trial 26 with value: 40.82597815818267.


Fold 10 MAE: 62.78099025800325
Mean MAE for all Folds: 62.639220692164464
Optimization complete!


In [15]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

test1 = pipin.drop_columns(test1)
test2 = pipin.drop_columns(test2)
test3 = pipin.drop_columns(test3)

In [16]:
pred1 = pd.DataFrame(predictor1.predict(test1), columns=["pv_measurement"])
pred2 = pd.DataFrame(predictor2.predict(test2), columns=["pv_measurement"])
pred3 = pd.DataFrame(predictor3.predict(test3), columns=["pv_measurement"])
negatives_pred1 = pred1[pred1["pv_measurement"] < 0]
negatives_pred2 = pred2[pred2["pv_measurement"] < 0]
negatives_pred3 = pred3[pred3["pv_measurement"] < 0]
neg = pd.concat([negatives_pred1, negatives_pred2, negatives_pred3])
neg.shape

(0, 1)

In [17]:
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv('submissions/XGB_RF_143_pipeline.csv')

In [18]:
diff = pipin.compare_mae(final_prediction)
diff

94.58406527433658

In [19]:
# predictor.evaluate(df1, silent=True)